In [2]:
import pickle

images, pca_features, pca = pickle.load(open('features_caltech101.p', 'rb'))

for i, f in list(zip(images, pca_features))[0:5]:
    print("image: %s, features: %0.2f,%0.2f,%0.2f,%0.2f... "%(i, f[0], f[1], f[2], f[3]))

image: 101_ObjectCategories/lamp/image_0006.jpg, features: -6.93,-6.46,-0.16,6.74... 
image: 101_ObjectCategories/lamp/image_0047.jpg, features: 0.24,0.25,-9.59,5.22... 
image: 101_ObjectCategories/lamp/image_0058.jpg, features: -11.10,-0.77,3.52,-13.11... 
image: 101_ObjectCategories/lamp/image_0022.jpg, features: -2.71,-5.86,-2.41,2.70... 
image: 101_ObjectCategories/lamp/image_0005.jpg, features: -8.87,-6.02,-4.04,2.85... 


In [7]:
num_images = 10000

if len(images) > num_images:
    sort_order = sorted(random.sample(xrange(len(images)), num_images))
    images = [images[i] for i in sort_order]
    pca_features = [pca_features[i] for i in sort_order]

In [2]:

from igraph import *

kNN = 30

graph = Graph(directed=True)
graph.add_vertices(len(images))

for i in tqdm(range(len(images))):
    distances = [ distance.cosine(pca_features[i], feat) for feat in pca_features ]
    idx_kNN = sorted(range(len(distances)), key=lambda k: distances[k])[1:kNN+1]
    for j in idx_kNN:
        graph.add_edge(i, j, weight=distances[j])
    
summary(graph)

DeprecationWarning: To avoid name collision with the igraph project, this visualization library has been renamed to 'jgraph'. Please upgrade when convenient.

In [ ]:
pickle.dump([images, graph], open('data/graph_caltech101_30knn.p', 'wb'))
#images, graph = pickle.load(open('../data/graph_caltech101_30knn.p', 'r'))

In [ ]:
def get_concatenated_images(indexes, thumb_height):
    thumbs = []
    for idx in indexes:
        img = Image.open(images[idx])
        img = img.convert('RGB')
        img = img.resize((int(img.width * thumb_height / img.height), thumb_height))
        thumbs.append(img)
    concat_image = np.concatenate([np.asarray(t) for t in thumbs], axis=1)
    return concat_image

In [ ]:
# pick two random indices
idx1 = int(len(images) * random.random())
idx2 = int(len(images) * random.random())

# run get_shortest_paths
path = graph.get_shortest_paths(idx1, to=idx2, mode=OUT, output='vpath', weights='weight')[0]

# retrieve the images, concatenate into one, and display them
results_image = get_concatenated_images(path, 200)
matplotlib.pyplot.figure(figsize = (16,12))
imshow(results_image)